In [0]:
import tensorflow as tf

In [0]:
layers_in_block = {'DenseNet-121': [6, 12, 24, 16],
                   'DenseNet-169': [6, 12, 32, 32],
                   'DenseNet-201': [6, 12, 48, 32],
                   'DenseNet-265': [6, 12, 64, 48]}

class DenseNet_classification(tf.keras.layers.Layer):
  def __init__(self, pooling='avg', classes=1):
    super(DenseNet_classification, self).__init__()
    self.classes = classes
    self.pooling = pooling

    self.avg_pooling = tf.keras.layers.GlobalAveragePooling2D()
    self.max_pooling = tf.keras.layers.GlobalMaxPooling2D()

    self.sigmoid_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.sigmoid)
    self.softmax_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.softmax)


  def call(self, inputs):
    if self.pooling == 'avg':
      x = self.avg_pooling(inputs)
    elif self.pooling == 'max':
      x = self.max_pooling(inputs)

    if self.classes == 1:
      x = self.sigmoid_fc(x)
    else:
      x = self.softmax_fc(x)
      
    return x

def make_dense_layer(pooling='avg', classes=1):
  res_dense = tf.keras.Sequential()
  res_dense.add(DenseNet_classification(pooling=pooling, classes=classes))
  return res_dense

In [0]:
class bottle_building_block(tf.keras.layers.Layer):
  def __init__(self, k=None):
    super(bottle_building_block, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters=k*4,
                                        kernel_size=(1, 1),
                                        padding='same',
                                        kernel_initializer='he_normal')
    self.bn1 = tf.keras.layers.BatchNormalization()
    
    self.conv2 = tf.keras.layers.Conv2D(filters=k,
                                        kernel_size=(3, 3),
                                        padding='same',
                                        kernel_initializer='he_normal')
    self.bn2 = tf.keras.layers.BatchNormalization()

  def call(self, inputs, training=None):
    x = self.bn1(inputs, training=training)
    x = tf.nn.relu(x)
    x = self.conv1(x)

    x = self.bn2(x, training=training)
    x = tf.nn.relu(x)
    x = self.conv2(x)

    return x

class transition_block(tf.keras.layers.Layer):
  def __init__(self, output_channels):
    super(transition_block, self).__init__()

    self.conv1 = tf.keras.layers.Conv2D(filters=output_channels,
                                        kernel_size=(1, 1),
                                        padding='same',
                                        kernel_initializer='he_normal')
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.avg_pooling = tf.keras.layers.AveragePooling2D(pool_size=(2,2),
                                                        strides=2,
                                                        padding="same")
    
  def call(self, inputs, training=None):
    x = self.bn1(inputs, training=training)
    x = tf.nn.relu(x)
    x = self.conv1(x)

    x = self.avg_pooling(x)

    return x

In [0]:
class DenseBlock(tf.keras.layers.Layer):
  def __init__(self, layer_block, growth_rate):
    super(DenseBlock, self).__init__()
    self.growth_rate = growth_rate
    self.layer_block = layer_block
    self.features_list = []

  def _make_layer(self, x):
    y = bottle_building_block(self.growth_rate)(x)
    self.features_list.append(y)
    y = tf.concat(self.features_list, axis=-1)
    return y

  def call(self, inputs):
    self.features_list.append(inputs)
    x = self._make_layer(inputs)
    for i in range(1, self.layer_block):
      x = self._make_layer(x)
    self.features_list.clear()

    return x

In [0]:
class DenseNet_model(tf.keras.Model):
  def __init__(self, num_init_features=32, layer_block='DenseNet-121', growth_rate=12, \
                reduction=0.5, classes=1, pooling='avg'):
    super(DenseNet_model, self).__init__()

    ## kernel_size = 3일 경우 zeropadding사용.
    # 따로 dropout 안넣어줌. 필요하면 사용. 대신 gradient/vanshied 완화로 kernel_initializer 넣어줌.
    self.conv1 = tf.keras.layers.Conv2D(filters=growth_rate*4,
                                        kernel_size=(7, 7),
                                        strides=2,
                                        padding="same",
                                        kernel_initializer='he_normal')
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.max_pooling = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                          strides=2,
                                          padding="same")
    self.num_channels = num_init_features  
    self.dense_block1 = DenseBlock(layers_in_block[layer_block][0], growth_rate)
    self.num_channels += growth_rate * layers_in_block[layer_block][0]
    self.num_channels = reduction * self.num_channels  
    self.transition_block1 = transition_block(int(self.num_channels))

    self.dense_block2 = DenseBlock(layers_in_block[layer_block][1], growth_rate)
    self.num_channels += growth_rate * layers_in_block[layer_block][1]
    self.num_channels = int(reduction * self.num_channels)
    self.transition_block2 = transition_block(int(self.num_channels))

    self.dense_block3 = DenseBlock(layers_in_block[layer_block][2], growth_rate)
    self.num_channels += growth_rate * layers_in_block[layer_block][2]
    self.num_channels = int(reduction * self.num_channels)
    self.transition_block3 = transition_block(int(self.num_channels))

    self.dense_block4 = DenseBlock(layers_in_block[layer_block][3], growth_rate)

    self.fc = make_dense_layer(pooling, classes)

  def call(self, inputs, training=None):
    x = self.conv1(inputs)
    x = self.bn1(x, training=training)
    x = tf.nn.relu(x)
    x = self.max_pooling(x)

    x = self.dense_block1(x)
    x = self.transition_block1(x)
    x = self.dense_block2(x)
    x = self.transition_block2(x)
    x = self.dense_block3(x)
    x = self.transition_block3(x)
    x = self.dense_block4(x)
    
    x = self.fc(x)

    return x

In [0]:
def DenseNet_121():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-121', growth_rate=32, reduction=0.5, classes=1, pooling='avg')

def DenseNet_169():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-169', growth_rate=32, reduction=0.5, classes=1, pooling='avg')

def DenseNet_201():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-201', growth_rate=32, reduction=0.5,classes=1, pooling='avg')

def DenseNet_265():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-265', growth_rate=32, reduction=0.5,classes=1, pooling='avg')